# Smoking is bad

In [ ]:
# TODO: install all the necessary packages
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.formula.api import probit
from pathlib import Path




In [ ]:
# TODO: load the data using the pathlib library to create the path to your data

path = Path('data/smoking_data.csv')
smoke_df = pd.read_csv(path)
print(smoke_df)


In [ ]:
# TODO: what are the dimensions of the data frame?
smoke_df.shape

In [ ]:
# TODO: use .describe() on your data frame: what do you notice about the string columns?
smoke_df.describe()

"""
The string columns dissapear as they are not numeric.
"""

In [ ]:
# TODO: do not forget to check the null values

# 1. See how many nulls
print(smoke_df.isnull().sum())

# 2. See percentage
print(smoke_df.isnull().sum() / len(smoke_df) * 100)

# 3. Check patterns
print(smoke_df[smoke_df['salary'].isnull()]['smoker'].value_counts())

In [ ]:
# TODO: prepare your data frame so that you can work with

smoke_df['smokes'] = (smoke_df['smoker'] == 'Yes').astype(int)
smoke_df['alive'] = (smoke_df['outcome'] == 'Alive').astype(int)
smoke_df['female'] = (smoke_df['gender'] == 'female').astype(int)


smoke_df.head()

## Smoking analysis
Let's first check what your cousin did.

In [ ]:
# TODO: Reimplement the small code snippet described in the problem set instructions.

smoke_df.groupby(["smokes"]).agg(prob_alive = ("alive", np.mean))

You should get the same results.

# Is smoking really good?
Now it is up to you to run the analysis further and check what could be underlying your cousings results.

In [ ]:
# Select only numeric columns
numeric_smoke_df = smoke_df.select_dtypes(include=['number'])
print(numeric_smoke_df)

In [ ]:
#Firstly trying to explain whether you're dead or alive.

import seaborn as sns

# Create subplots for each variable
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Survival by smoker
smoke_df.groupby('smoker')['alive'].mean().plot(kind='bar', ax=axes[0,0])
axes[0,0].set_title('Survival by Smoking')
axes[0,0].set_ylabel('Survival Probability')

# Age distribution
smoke_df.boxplot(column='age', by='smoker', ax=axes[0,1])
axes[0,1].set_title('Age Distribution by Smoking')

# Gender distribution
smoke_df.groupby(['smoker', 'female'])['alive'].count().unstack().plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('Count by Smoking and Gender')

# Salary by smoking
smoke_df.boxplot(column='salary', by='smoker', ax=axes[1,1])
axes[1,1].set_title('Salary by Smoking')

plt.tight_layout()
plt.show()



In [ ]:
#Estimating probit of being alive just using smoking
model = probit('alive ~ smokes', data=numeric_smoke_df.dropna())
results = model.fit()
print(results.summary())


In [ ]:
# Estimate probit model
model = probit('alive ~ smokes + age + female + salary', 
               data=numeric_smoke_df.dropna())
results = model.fit()

# Print results
print(results.summary())

Using just smoking as an explanatory variable for probability of being alive explains very little variation (Pseudo R^2 = 1%)

Adding factors such as ages and being a female we are able to explain a lot more variation (37%) and the impact of smoking makes it 10% less likely that you are Alive. It may be that smoking was correlated with other factors.

In [ ]:
print(numeric_smoke_df.corr())